In [ ]:
from peaceful_pie.unity_comms import UnityComms
from peaceful_pie.unity_comms import UnityComms

In [ ]:
def unity_comms(port: int):
    unity_comms = UnityComms(port)
    return unity_comms
port = 5000  # Replace with your desired port number
unity_comms_instance = unity_comms(port)

In [ ]:
unity_comms_instance 
unity_comms = unity_comms_instance

In [ ]:
import numpy as np

def calculate_final_location(point):
    # Retrieve the normalized distances for the specified point
    distances = full_normalized_distance[point]

    # Set the initial position of the wrist
    initial_wrist_x = 2442.86  # Choose your desired x-coordinate
    initial_wrist_y = -18.806  # Choose your desired y-coordinate
    initial_wrist_z = -473.12  # Choose your desired z-coordinate

    # Initialize the position of the specified point
    point_x = initial_wrist_x
    point_y = initial_wrist_y
    point_z = initial_wrist_z

    # Update the position of the specified point based on the normalized distances
    for distance in distances:
        point_x += distance  # Update the x-coordinate
        point_y += distance  # Update the y-coordinate
        point_z += distance  # Update the z-coordinate

    # Calculate the final location of the specified point
    final_location = np.array([point_x, point_y, point_z])

    return final_location


In [ ]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
import cv2
from mediapipe.tasks.python import vision
import threading

# STEP 2: Create a Hands object.
mp_hands = mp.solutions.hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# STEP 3: Open the video capture.
cap = cv2.VideoCapture(0)


In [ ]:
import numpy as np

In [ ]:
full_normalized_distance = []
full_real_distance = []
full_final_location = []
# Define the indices for the hand landmarks
WRIST = 0
_     = 1
THUMB_MCP = 2
THUMB_DIP = 3
THUMB_TIP = 4
INDEX_FINGER_MCP = 5
INDEX_FINGER_PIP = 6
INDEX_FINGER_DIP = 7
INDEX_FINGER_TIP = 8
MIDDLE_FINGER_MCP = 9
MIDDLE_FINGER_PIP = 10
MIDDLE_FINGER_DIP = 11
MIDDLE_FINGER_TIP = 12
RING_FINGER_MCP = 13
RING_FINGER_PIP = 14
RING_FINGER_DIP = 15
RING_FINGER_TIP = 16
PINKY_MCP = 17
PINKY_PIP = 18
PINKY_DIP = 19
PINKY_TIP = 20

# Set the initial position of the wrist
initial_wrist_x = 2442.86  # Choose your desired x-coordinate
initial_wrist_y = -18.806  # Choose your desired y-coordinate
initial_wrist_z = -473.12  # Choose your desired z-coordinate
initial_wrist = np.array((initial_wrist_x, initial_wrist_y, initial_wrist_z))


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the BGR frame to RGB.
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Create a copy of the image for display.
    display_image = image.copy()

    # Flip the image vertically for an upside-down view.
    image = cv2.flip(image, 0)

    # Flip the image horizontally for the original orientation.
    image = cv2.flip(image, 1)

    # Process the image with the Hands model.
    results = mp_hands.process(image)

    # Draw hand landmarks on the image.
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(
                image,
                hand_landmarks,
                mp.solutions.hands.HAND_CONNECTIONS
            )

            # Reset the lists for each hand_landmarks
            real_distance = []
            distances = []

            # Calculate distances between each pair of landmark points
            num_landmarks = len(hand_landmarks.landmark)
            for i in range(num_landmarks):
                for j in range(i + 1, num_landmarks):
                    point1 = np.array((hand_landmarks.landmark[i].x * image.shape[1],
                                       hand_landmarks.landmark[i].y * image.shape[0],
                                       hand_landmarks.landmark[i].z * image.shape[2]))
                    point2 = np.array((hand_landmarks.landmark[j].x * image.shape[1],
                                       hand_landmarks.landmark[j].y * image.shape[0],
                                       hand_landmarks.landmark[j].z * image.shape[2]))
                    distance = np.linalg.norm(point1 - point2)
                    real_distance.append(distance)
                    normalized_distance = (distance / np.linalg.norm(initial_wrist)*100)
                    distances.append(normalized_distance)

            full_real_distance.append(real_distance)
            full_normalized_distance.append(distances)

    # Check if the full_normalized_distance list is not empty
    if len(full_normalized_distance) > 0:
        # Define the calculate_final_location function
        def calculate_final_location(point):
            # Check if the point index is within range
            if point < len(full_normalized_distance):
                # Retrieve the normalized distances for the specified point
                distances = full_normalized_distance[point]

                # Set the initial position of the wrist
                initial_wrist_x = 2442.86  # Choose your desired x-coordinate
                initial_wrist_y = -18.806  # Choose your desired y-coordinate
                initial_wrist_z = -473.12  # Choose your desired z-coordinate

                # Initialize the position of the specified point
                point_x = initial_wrist_x
                point_y = initial_wrist_y
                point_z = initial_wrist_z

                # Update the position of the specified point based on the normalized distances
                for distance in distances:
                    point_x += distance  # Update the x-coordinate
                    point_y += distance  # Update the y-coordinate
                    point_z += distance  # Update the z-coordinate

                # Calculate the final location of the specified point
                final_location = np.array([point_x, point_y, point_z])

                return final_location

        # Calculate the final location of all the landmarks
        final_locations = []
        for i in range(21):
            final_locations.append(calculate_final_location(i))

        # Check if the index finger tip location is not None
        if final_locations[INDEX_FINGER_TIP] is not None:
            x = final_locations[INDEX_FINGER_TIP][0]
            print(f'this is x: {x}')
            y = final_locations[INDEX_FINGER_TIP][1]
            print(f'this is y: {y}')
            z = final_locations[INDEX_FINGER_TIP][2]
            print(f'this is z: {z}')

            # Check if x, y, and z are not None and greater than 0
            if x is not None and y is not None and z is not None and x > 0 and y > 0 and z > 0:
                unity_comms.SetPosition_index_tip(x, y, z)

    display_image = cv2.cvtColor(display_image, cv2.COLOR_RGB2BGR)
    # Display the image with hand landmarks.
    cv2.imshow('MediaPipe Hands', display_image)

    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
def calculate_final_location(point):
    if point < 0 or point >= len(full_normalized_distance):
        return None
    distances = full_normalized_distance[point]
    initial_wrist_x = 2442.86
    initial_wrist_y = 1551.15
    initial_wrist_z = 5.0
    final_location = [initial_wrist_x + distances[0], initial_wrist_y + distances[1], initial_wrist_z + distances[2]]
    return final_location


In [21]:
full_normalized_distance = []
full_real_distance = []
full_final_location = []
# Define the indices for the hand landmarks
WRIST = 0
_     = 1
THUMB_MCP = 2
THUMB_DIP = 3
THUMB_TIP = 4
INDEX_FINGER_MCP = 5
INDEX_FINGER_PIP = 6
INDEX_FINGER_DIP = 7
INDEX_FINGER_TIP = 8
MIDDLE_FINGER_MCP = 9
MIDDLE_FINGER_PIP = 10
MIDDLE_FINGER_DIP = 11
MIDDLE_FINGER_TIP = 12
RING_FINGER_MCP = 13
RING_FINGER_PIP = 14
RING_FINGER_DIP = 15
RING_FINGER_TIP = 16
PINKY_MCP = 17
PINKY_PIP = 18
PINKY_DIP = 19
PINKY_TIP = 20

# Set the initial position of the wrist
initial_wrist_x = 2442.86  # Choose your desired x-coordinate
initial_wrist_y = -18.806  # Choose your desired y-coordinate
initial_wrist_z = -473.12  # Choose your desired z-coordinate
initial_wrist = np.array((initial_wrist_x, initial_wrist_y, initial_wrist_z))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the BGR frame to RGB.
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Create a copy of the image for display.
    display_image = image.copy()

    # Flip the image vertically for an upside-down view.
    image = cv2.flip(image, 0)

    # Flip the image horizontally for the original orientation.
    image = cv2.flip(image, 1)

    # Process the image with the Hands model.
    results = mp_hands.process(image)

    # Draw hand landmarks on the image.
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(
                image,
                hand_landmarks,
                mp.solutions.hands.HAND_CONNECTIONS
            )

            # Reset the lists for each hand_landmarks
            real_distance = []
            distances = []

            # Calculate distances between each pair of landmark points
            num_landmarks = len(hand_landmarks.landmark)
            for i in range(num_landmarks):
                for j in range(i + 1, num_landmarks):
                    point1 = np.array((hand_landmarks.landmark[i].x * image.shape[1],
                                       hand_landmarks.landmark[i].y * image.shape[0],
                                       hand_landmarks.landmark[i].z * image.shape[2]))
                    point2 = np.array((hand_landmarks.landmark[j].x * image.shape[1],
                                       hand_landmarks.landmark[j].y * image.shape[0],
                                       hand_landmarks.landmark[j].z * image.shape[2]))
                    distance = np.linalg.norm(point1 - point2)
                    real_distance.append(distance)
                    normalized_distance = distance / np.linalg.norm(initial_wrist)
                    distances.append(normalized_distance)

            full_real_distance.append(real_distance)
            full_normalized_distance.append(distances)

    # Check if the full_normalized_distance list is not empty
    if len(full_normalized_distance) > 0:



        # Define the calculate_final_location function
        def calculate_final_location(point):
            # Check if the point index is within range
            if point < len(full_normalized_distance):
                # Retrieve the normalized distances for the specified point
                distances = full_normalized_distance[point]

                # Set the initial position of the wrist
                initial_wrist_x = 2442.86  # Choose your desired x-coordinate
                initial_wrist_y = -18.806  # Choose your desired y-coordinate
                initial_wrist_z = -473.12  # Choose your desired z-coordinate

                # Initialize the position of the specified point
                point_x = initial_wrist_x
                point_y = initial_wrist_y
                point_z = initial_wrist_z

                # Update the position of the specified point based on the normalized distances
                for distance in distances:
                    point_x += distance  # Update the x-coordinate
                    point_y += distance  # Update the y-coordinate
                    point_z += distance  # Update the z-coordinate

                # Calculate the final location of the specified point
                final_location = np.array((point_x, point_y, point_z))

                return final_location

        # Calculate the final location of all the landmarks
        final_locations = []
        for i in range(21):
            final_locations.append(calculate_final_location(i))

        full_final_location.append(final_locations)

        # Check if the index finger tip location is not None
        index_finger_tip_location = calculate_final_location(INDEX_FINGER_TIP)
        if index_finger_tip_location is not None:
            x = index_finger_tip_location[0]

            print(f'location of x is {x}')
            y = index_finger_tip_location[1]
            print(f'location of y is {y}')

            z = index_finger_tip_location[2]
            print(f'location of z is {z}')
            if x != None and y != None and z != None:
                print('sending data')
                print(f'location of x is {x}')
                print(f'location of y is {y}')
                print(f'location of z is {z}')
                x= float(x)
                y= float(y)
                z= float(z)
                arr = (x,y,z)
                print(arr)
                unity_comms.SetPosition_index_tip(arr[0],arr[1],arr[2])

    display_image = cv2.cvtColor(display_image, cv2.COLOR_RGB2BGR)
    # Display the image with hand landmarks.
    # Display the image with hand landmarks.
    cv2.imshow('MediaPipe Hands', display_image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break



location of x is 2452.6959728737493
location of y is -8.970027126250807
location of z is -463.2840271262508
sending data
location of x is 2452.6959728737493
location of y is -8.970027126250807
location of z is -463.2840271262508
(2452.6959728737493, -8.970027126250807, -463.2840271262508)


TypeError: __call__() takes from 1 to 3 positional arguments but 4 were given

In [ ]:
full_normalized_distance = []
full_real_distance = []
full_final_location = []
# Define the indices for the hand landmarks
WRIST = 0
_     = 1
THUMB_MCP = 2
THUMB_DIP = 3
THUMB_TIP = 4
INDEX_FINGER_MCP = 5
INDEX_FINGER_PIP = 6
INDEX_FINGER_DIP = 7
INDEX_FINGER_TIP = 8
MIDDLE_FINGER_MCP = 9
MIDDLE_FINGER_PIP = 10
MIDDLE_FINGER_DIP = 11
MIDDLE_FINGER_TIP = 12
RING_FINGER_MCP = 13
RING_FINGER_PIP = 14
RING_FINGER_DIP = 15
RING_FINGER_TIP = 16
PINKY_MCP = 17
PINKY_PIP = 18
PINKY_DIP = 19
PINKY_TIP = 20

# Set the initial position of the wrist
initial_wrist_x = 2442.86  # Choose your desired x-coordinate
initial_wrist_y = -18.806  # Choose your desired y-coordinate
initial_wrist_z = -473.12  # Choose your desired z-coordinate
initial_wrist = np.array((initial_wrist_x, initial_wrist_y, initial_wrist_z))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the BGR frame to RGB.
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Create a copy of the image for display.
    display_image = image.copy()

    # Flip the image vertically for an upside-down view.
    image = cv2.flip(image, 0)

    # Flip the image horizontally for the original orientation.
    image = cv2.flip(image, 1)

    # Process the image with the Hands model.
    results = mp_hands.process(image)

    # Draw hand landmarks on the image.
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(
                image,
                hand_landmarks,
                mp.solutions.hands.HAND_CONNECTIONS
            )

            # Reset the lists for each hand_landmarks
            real_distance = []
            distances = []

            # Calculate distances between each pair of landmark points
            num_landmarks = len(hand_landmarks.landmark)
            for i in range(num_landmarks):
                for j in range(i + 1, num_landmarks):
                    point1 = np.array((hand_landmarks.landmark[i].x * image.shape[1],
                                       hand_landmarks.landmark[i].y * image.shape[0],
                                       hand_landmarks.landmark[i].z * image.shape[2]))
                    point2 = np.array((hand_landmarks.landmark[j].x * image.shape[1],
                                       hand_landmarks.landmark[j].y * image.shape[0],
                                       hand_landmarks.landmark[j].z * image.shape[2]))
                    distance = np.linalg.norm(point1 - point2)
                    real_distance.append(distance)
                    normalized_distance = distance / np.linalg.norm(initial_wrist)
                    distances.append(normalized_distance)

            full_real_distance.append(real_distance)
            full_normalized_distance.append(distances)

    # Check if the full_normalized_distance list is not empty
    if len(full_normalized_distance) > 0:



        # Define the calculate_final_location function
        def calculate_final_location(point):
            # Check if the point index is within range
            if point < len(full_normalized_distance):
                # Retrieve the normalized distances for the specified point
                distances = full_normalized_distance[point]

                # Set the initial position of the wrist
                initial_wrist_x = 2442.86  # Choose your desired x-coordinate
                initial_wrist_y = -18.806  # Choose your desired y-coordinate
                initial_wrist_z = -473.12  # Choose your desired z-coordinate

                # Initialize the position of the specified point
                point_x = initial_wrist_x
                point_y = initial_wrist_y
                point_z = initial_wrist_z

                # Update the position of the specified point based on the normalized distances
                for distance in distances:
                    point_x += distance  # Update the x-coordinate
                    point_y += distance  # Update the y-coordinate
                    point_z += distance  # Update the z-coordinate

                # Calculate the final location of the specified point
                final_location = np.array((point_x, point_y, point_z))

                return final_location

        # Calculate the final location of all the landmarks
        final_locations = []
        for i in range(21):
            final_locations.append(calculate_final_location(i))

        full_final_location.append(final_locations)

        # Check if the index finger tip location is not None
        index_finger_tip_location = calculate_final_location(INDEX_FINGER_TIP)
        if index_finger_tip_location is not None:
            x = index_finger_tip_location[0]

            print(f'location of x is {x}')
            y = index_finger_tip_location[1]
            print(f'location of y is {y}')

            z = index_finger_tip_location[2]
            print(f'location of z is {z}')
            if x != None and y != None and z != None:
                print('sending data')
                print(f'location of x is {x}')
                print(f'location of y is {y}')
                print(f'location of z is {z}')
                unity_comms.SetPosition_index_tip(x, y, z)

    display_image = cv2.cvtColor(display_image, cv2.COLOR_RGB2BGR)
    # Display the image with hand landmarks.
    # Display the image with hand landmarks.
    cv2.imshow('MediaPipe Hands', display_image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break



In [ ]:
import numpy as np

In [ ]:
x=np.random.randint(0, 100)
y=np.random.randint(0, 100)
z=np.random.randint(0, 100)
arr = np.array((x, y, z))

In [ ]:
arr
print(arr)

In [ ]:
lst

In [ ]:
x = 3017.2312688081693
y = 555.5652688081691
z = 101.25126880816869

unity_comms.SetPosition_index_tip((x), (y), (z))


In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
full_normalized_distance = []
full_real_distance = []
full_final_location = []
# Define the indices for the hand landmarks
WRIST = 0
_     = 1
THUMB_MCP = 2
THUMB_DIP = 3
THUMB_TIP = 4
INDEX_FINGER_MCP = 5
INDEX_FINGER_PIP = 6
INDEX_FINGER_DIP = 7
INDEX_FINGER_TIP = 8
MIDDLE_FINGER_MCP = 9
MIDDLE_FINGER_PIP = 10
MIDDLE_FINGER_DIP = 11
MIDDLE_FINGER_TIP = 12
RING_FINGER_MCP = 13
RING_FINGER_PIP = 14
RING_FINGER_DIP = 15
RING_FINGER_TIP = 16
PINKY_MCP = 17
PINKY_PIP = 18
PINKY_DIP = 19
PINKY_TIP = 20

# Set the initial position of the wrist
initial_wrist_x = 2442.86  # Choose your desired x-coordinate
initial_wrist_y = -18.806  # Choose your desired y-coordinate
initial_wrist_z = -473.12  # Choose your desired z-coordinate
initial_wrist = np.array((initial_wrist_x, initial_wrist_y, initial_wrist_z))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the BGR frame to RGB.
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Create a copy of the image for display.
    display_image = image.copy()

    # Flip the image vertically for an upside-down view.
    image = cv2.flip(image, 0)

    # Flip the image horizontally for the original orientation.
    image = cv2.flip(image, 1)

    # Process the image with the Hands model.
    results = mp_hands.process(image)

    # Draw hand landmarks on the image.
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(
                image,
                hand_landmarks,
                mp.solutions.hands.HAND_CONNECTIONS
            )

            # Reset the lists for each hand_landmarks
            real_distance = []
            distances = []

            # Calculate distances between each pair of landmark points
            num_landmarks = len(hand_landmarks.landmark)
            for i in range(num_landmarks):
                for j in range(i+1, num_landmarks):
                    point1 = np.array((hand_landmarks.landmark[i].x * image.shape[1],
                                       hand_landmarks.landmark[i].y * image.shape[0],
                                       hand_landmarks.landmark[i].z * image.shape[2]))
                    point2 = np.array((hand_landmarks.landmark[j].x * image.shape[1],
                                       hand_landmarks.landmark[j].y * image.shape[0],
                                       hand_landmarks.landmark[j].z * image.shape[2]))
                    distance = np.linalg.norm(point1 - point2)
                    real_distance.append(distance)
                    normalized_distance = distance / np.linalg.norm(initial_wrist)
                    distances.append(normalized_distance)

            full_real_distance.append(real_distance)
            full_normalized_distance.append(distances)

    # Check if the full_normalized_distance list is not empty
    if len(full_normalized_distance) > 0:
        # Define the calculate_final_location function
        def calculate_final_location(point):
            # Check if the point index is within range
            if point < len(full_normalized_distance):
                # Retrieve the normalized distances for the specified point
                distances = full_normalized_distance[point]

                # Set the initial position of the wrist
                initial_wrist_x = 2442.86  # Choose your desired x-coordinate
                initial_wrist_y = -18.806  # Choose your desired y-coordinate
                initial_wrist_z = -473.12  # Choose your desired z-coordinate

                # Initialize the position of the specified point
                point_x = initial_wrist_x
                point_y = initial_wrist_y
                point_z = initial_wrist_z

                # Update the position of the specified point based on the normalized distances
                for distance in distances:
                    point_x += distance  # Update the x-coordinate
                    point_y += distance  # Update the y-coordinate
                    point_z += distance  # Update the z-coordinate

                # Calculate the final location of the specified point
                final_location = np.array([point_x, point_y, point_z])

                return final_location
            
        index_finger_tip_location = calculate_final_location(INDEX_FINGER_TIP)
        print(index_finger_tip_location)
        if index_finger_tip_location is not None:
            x=index_finger_tip_location[0]
            y=index_finger_tip_location[1]
            z=index_finger_tip_location[2]
            if x,y,z>0:
                unity_comms.SetPosition_index_tip(index_finger_tip_location[0], index_finger_tip_location[1], index_finger_tip_location[2])


        # # Calculate the final location of all the landmarks
        # final_locations = []
        # for i in range(21):
        #     final_locations.append(calculate_final_location(i))

        # full_final_location.append(final_locations)
        # if len(full_final_location)>0:
        #     print(full_final_location[0])

            
    display_image = cv2.cvtColor(display_image, cv2.COLOR_RGB2BGR)

    # Display the image with hand landmarks.
    cv2.imshow('MediaPipe Hands', display_image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


In [ ]:
full_final_location[]

In [ ]:

# Find the index of the INDEX_FINGER_TIP in the hand landmarks
index_finger_tip_index = INDEX_FINGER_TIP

# Get the normalized distances from the wrist to all other hand landmarks
wrist_distances = full_normalized_distance[WRIST]

# Get the normalized distances from the INDEX_FINGER_TIP to all other hand landmarks
index_finger_tip_distances = full_normalized_distance[index_finger_tip_index]

# Calculate the difference in distances between the wrist and INDEX_FINGER_TIP for each hand landmark
distance_differences = [abs(wrist_distance - index_finger_tip_distance)
                        for wrist_distance, index_finger_tip_distance
                        in zip(wrist_distances, index_finger_tip_distances)]

# Sort the hand landmarks based on the difference in distances
sorted_landmarks = [landmark for _, landmark
                    in sorted(zip(distance_differences, hand_landmarks.landmark),
                              key=lambda x: x[0])]

# Navigate using the sorted_landmarks, where the closest landmarks are first in the list


In [ ]:
full_normalized_distance[8].shape

In [ ]:
distance_differences

In [ ]:
import numpy as np

def calculate_final_location(point):
    # Retrieve the normalized distances for the specified point
    distances = full_normalized_distance[point]

    # Set the initial position of the wrist
    initial_wrist_x = 2442.86  # Choose your desired x-coordinate
    initial_wrist_y = -18.806  # Choose your desired y-coordinate
    initial_wrist_z = -473.12  # Choose your desired z-coordinate

    # Initialize the position of the specified point
    point_x = initial_wrist_x
    point_y = initial_wrist_y
    point_z = initial_wrist_z

    # Update the position of the specified point based on the normalized distances
    for distance in distances:
        point_x += distance  # Update the x-coordinate
        point_y += distance  # Update the y-coordinate
        point_z += distance  # Update the z-coordinate

    # Calculate the final location of the specified point
    final_location = np.array([point_x, point_y, point_z])

    return final_location


In [ ]:
index_finger_tip_location = calculate_final_location(RING_FINGER_PIP)
print("Final Location of Index Finger Tip:", index_finger_tip_location)


In [ ]:
import numpy as np

# Retrieve the normalized distances for the index finger tip
index_finger_tip_distances = full_normalized_distance[8]

# Set the initial position of the wrist
initial_wrist_x = 2442.86  # Choose your desired x-coordinate
initial_wrist_y = -18.806  # Choose your desired y-coordinate
initial_wrist_z = -473.12  # Choose your desired z-coordinate

# Initialize the position of the index finger tip
index_finger_tip_x = initial_wrist_x
index_finger_tip_y = initial_wrist_y
index_finger_tip_z = initial_wrist_z

# Update the position of the index finger tip based on the normalized distances
for distance in index_finger_tip_distances:
    index_finger_tip_x += distance  # Update the x-coordinate
    index_finger_tip_y += distance  # Update the y-coordinate
    index_finger_tip_z += distance  # Update the z-coordinate

# Calculate the final location of the index finger tip
final_index_finger_tip = np.array([index_finger_tip_x, index_finger_tip_y, index_finger_tip_z])

# Print the final location
print("Final Location of Index Finger Tip:", final_index_finger_tip)


In [ ]:
import matplotlib.pyplot as plt

# Retrieve the normalized distances for the index finger tip
index_finger_tip_distances = full_normalized_distance[8]

# Set the initial position of the wrist
initial_wrist_x = 2442.86  # Choose your desired x-coordinate
initial_wrist_y = -18.806  # Choose your desired y-coordinate
initial_wrist_z = -473.12  # Choose your desired z-coordinate

# Initialize the position of the index finger tip
index_finger_tip_x = initial_wrist_x
index_finger_tip_y = initial_wrist_y
index_finger_tip_z = initial_wrist_z

# Lists to store the x, y, and z coordinates of the index finger tip
index_finger_tip_x_coords = [index_finger_tip_x]
index_finger_tip_y_coords = [index_finger_tip_y]
index_finger_tip_z_coords = [index_finger_tip_z]

# Update the position of the index finger tip based on the normalized distances
for distance in index_finger_tip_distances:
    index_finger_tip_x += distance  # Update the x-coordinate
    index_finger_tip_y += distance  # Update the y-coordinate
    index_finger_tip_z += distance  # Update the z-coordinate
    
    index_finger_tip_x_coords.append(index_finger_tip_x)
    index_finger_tip_y_coords.append(index_finger_tip_y)
    index_finger_tip_z_coords.append(index_finger_tip_z)

# Plot the movement of the index finger tip in 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.plot(index_finger_tip_x_coords, index_finger_tip_y_coords, index_finger_tip_z_coords)
ax.scatter(index_finger_tip_x_coords, index_finger_tip_y_coords, index_finger_tip_z_coords, c='r', marker='o')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Movement of Index Finger Tip')

plt.show()


In [ ]:
len(sorted_landmarks)

In [ ]:
full_normalized_distance = np.array(full_normalized_distance)

In [ ]:
full_normalized_distance.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(full_normalized_distance[:, 1])

In [ ]:
plt.plot(full_normalized_distance[0])

In [ ]:
num_frames = len(full_normalized_distance)
num_combinations = len(full_normalized_distance[0])

print("Shape of full_normalized_distance:", (num_frames, num_combinations))
print("Shape of full_real_distance:", (num_frames, num_combinations))


In [ ]:
full_normalized_distance=[]
full_real_distance=[]
WRIST = 0
THUMB_CMC = 1
THUMB_MCP = 2
THUMB_IP = 3
THUMB_TIP = 4
INDEX_FINGER_MCP = 5
INDEX_FINGER_PIP = 6
INDEX_FINGER_DIP = 7
INDEX_FINGER_TIP = 8
MIDDLE_FINGER_MCP = 9
MIDDLE_FINGER_PIP = 10
MIDDLE_FINGER_DIP = 11
MIDDLE_FINGER_TIP = 12
RING_FINGER_MCP = 13
RING_FINGER_PIP = 14
RING_FINGER_DIP = 15
RING_FINGER_TIP = 16
PINKY_MCP = 17
PINKY_PIP = 18
PINKY_DIP = 19
PINKY_TIP = 20


# Set the initial position of the wrist
initial_wrist_x = 2442.86  # Choose your desired x-coordinate
initial_wrist_y = -18.806  # Choose your desired y-coordinate
initial_wrist_z = -473.12  # Choose your desired z-coordinate
initial_wrist= (initial_wrist_x, initial_wrist_y, initial_wrist_z)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the BGR frame to RGB.
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Create a copy of the image for display.
    display_image = image.copy()

    # Flip the image vertically for an upside-down view.
    image = cv2.flip(image, 0)

    # Flip the image horizontally for the original orientation.
    image = cv2.flip(image, 1)
    
    # Process the image with the Hands model.
    results = mp_hands.process(image)

    # Draw hand landmarks on the image.
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(
                image,
                hand_landmarks,
                mp.solutions.hands.HAND_CONNECTIONS
            )


            real_distance=[]

            distances = []
            for landmark in hand_landmarks.landmark:
                x = landmark.x * image.shape[1]
                y = landmark.y * image.shape[0]
                z = landmark.z * image.shape[2]
                point = np.array((x, y, z))
                distance = np.linalg.norm(point - initial_wrist)
                real_distance.append(distance)
                full_real_distance.append(real_distance)
                normalized_distance = distance / np.linalg.norm(initial_wrist)
                distances.append(normalized_distance)     
                full_normalized_distance.append(distances)  


            # STEP 4: Send the normalized distances to Unity.
            if len(distances) == 21:
                index_tip_distance = distances[INDEX_FINGER_TIP]
                         
    display_image = cv2.cvtColor(display_image, cv2.COLOR_RGB2BGR)

    # Display the image with hand landmarks.
    cv2.imshow('MediaPipe Hands', display_image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break



In [ ]:
full_normalized_distance = np.array(full_normalized_distance)
full_real_distance = np.array(full_real_distance)

In [ ]:
full_normalized_distance[50][50]

In [ ]:
full_real_distance[0]

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
plt.plot(full_real_distance[0])

In [ ]:
plt.plot(full_normalized_distance[0])

In [ ]:

plt.plot(full_distance[:, INDEX_FINGER_TIP], label='Index finger tip')

In [ ]:
plt.plot(full_distance[0])
plt.show()
plt.plot(full_distance[1])
plt.show()
plt.plot(full_distance[2])
plt.show()

In [ ]:
full_distance[0]

In [ ]:
distances


In [ ]:
full_normalized_distance=[]
full_real_distance=[]
WRIST = 0
THUMB_CMC = 1
THUMB_MCP = 2
THUMB_IP = 3
THUMB_TIP = 4
INDEX_FINGER_MCP = 5
INDEX_FINGER_PIP = 6
INDEX_FINGER_DIP = 7
INDEX_FINGER_TIP = 8
MIDDLE_FINGER_MCP = 9
MIDDLE_FINGER_PIP = 10
MIDDLE_FINGER_DIP = 11
MIDDLE_FINGER_TIP = 12
RING_FINGER_MCP = 13
RING_FINGER_PIP = 14
RING_FINGER_DIP = 15
RING_FINGER_TIP = 16
PINKY_MCP = 17
PINKY_PIP = 18
PINKY_DIP = 19
PINKY_TIP = 20


# Set the initial position of the wrist
initial_wrist_x = 2442.86  # Choose your desired x-coordinate
initial_wrist_y = -18.806  # Choose your desired y-coordinate
initial_wrist_z = -473.12  # Choose your desired z-coordinate
initial_wrist= (initial_wrist_x, initial_wrist_y, initial_wrist_z)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the BGR frame to RGB.
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Create a copy of the image for display.
    display_image = image.copy()

    # Flip the image vertically for an upside-down view.
    image = cv2.flip(image, 0)

    # Flip the image horizontally for the original orientation.
    image = cv2.flip(image, 1)
    
    # Process the image with the Hands model.
    results = mp_hands.process(image)

    # Draw hand landmarks on the image.
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(
                image,
                hand_landmarks,
                mp.solutions.hands.HAND_CONNECTIONS
            )


            real_distance=[]

            distances = []
            for landmark in hand_landmarks.landmark:
                x = landmark.x * image.shape[1]
                y = landmark.y * image.shape[0]
                z = landmark.z * image.shape[2]
                point = np.array((x, y, z))
                distance = np.linalg.norm(point - initial_wrist)
                real_distance.append(distance)
                full_real_distance.append(real_distance)
                normalized_distance = distance / np.linalg.norm(initial_wrist)
                distances.append(normalized_distance)     
                full_normalized_distance.append(distances)  
                         


            # Calculate the distance of landmarks relative to the initial wrist position
            wrist_x, wrist_y, wrist_z = points[WRIST]  # Retrieve the wrist coordinates
            relative_points = [(x - wrist_x, y - wrist_y, z - wrist_z) for x, y, z in points]
            points = relative_points
            if len(points) > INDEX_FINGER_TIP:
                index_tip = points[INDEX_FINGER_TIP]
                normalized_tip = (index_tip[0] / image.shape[1], index_tip[1] / image.shape[0], index_tip[2] / image.shape[2])
                unity_comms.SetPosition_index_tip(x=normalized_tip[0], y=normalized_tip[1], z=normalized_tip[2])
                
                index_dip = points[INDEX_FINGER_DIP]
                normalized_dip = (index_dip[0] / image.shape[1], index_dip[1] / image.shape[0], index_dip[2] / image.shape[2])
                unity_comms.SetPosition_index_dip(x=normalized_dip[0], y=normalized_dip[1], z=normalized_dip[2])
                
                index_pip = points[INDEX_FINGER_PIP]
                normalized_pip = (index_pip[0] / image.shape[1], index_pip[1] / image.shape[0], index_pip[2] / image.shape[2])
                unity_comms.SetPosition_index_pip(x=normalized_pip[0], y=normalized_pip[1], z=normalized_pip[2])
                
                index_mcp = points[INDEX_FINGER_MCP]
                normalized_mcp = (index_mcp[0] / image.shape[1], index_mcp[1] / image.shape[0], index_mcp[2] / image.shape[2])
                unity_comms.SetPosition_index_mcp(x=normalized_mcp[0], y=normalized_mcp[1], z=normalized_mcp[2])
                
                middle_tip = points[MIDDLE_FINGER_TIP]
                normalized_tip = (middle_tip[0] / image.shape[1], middle_tip[1] / image.shape[0], middle_tip[2] / image.shape[2])
                unity_comms.SetPosition_middle_tip(x=normalized_tip[0], y=normalized_tip[1], z=normalized_tip[2])
                
                middle_dip = points[MIDDLE_FINGER_DIP]
                normalized_dip = (middle_dip[0] / image.shape[1], middle_dip[1] / image.shape[0], middle_dip[2] / image.shape[2])
                unity_comms.SetPosition_middle_dip(x=normalized_dip[0], y=normalized_dip[1], z=normalized_dip[2])
                
                middle_pip = points[MIDDLE_FINGER_PIP]
                normalized_pip = (middle_pip[0] / image.shape[1], middle_pip[1] / image.shape[0], middle_pip[2] / image.shape[2])
                unity_comms.SetPosition_middle_pip(x=normalized_pip[0], y=normalized_pip[1], z=normalized_pip[2])
                
                middle_mcp = points[MIDDLE_FINGER_MCP]
                normalized_mcp = (middle_mcp[0] / image.shape[1], middle_mcp[1] / image.shape[0], middle_mcp[2] / image.shape[2])
                unity_comms.SetPosition_middle_mcp(x=normalized_mcp[0], y=normalized_mcp[1], z=normalized_mcp[2])
                ring_tip = points[RING_FINGER_TIP]
                normalized_tip = (ring_tip[0] / image.shape[1], ring_tip[1] / image.shape[0], ring_tip[2] / image.shape[2])
                unity_comms.SetPosition_ring_tip(x=normalized_tip[0], y=normalized_tip[1], z=normalized_tip[2])
                ring_dip = points[RING_FINGER_DIP]
                normalized_dip = (ring_dip[0] / image.shape[1], ring_dip[1] / image.shape[0], ring_dip[2] / image.shape[2])
                unity_comms.SetPosition_ring_dip(x=normalized_dip[0], y=normalized_dip[1], z=normalized_dip[2])
                ring_pip = points[RING_FINGER_PIP]
                normalized_pip = (ring_pip[0] / image.shape[1], ring_pip[1] / image.shape[0], ring_pip[2] / image.shape[2])
                unity_comms.SetPosition_ring_pip(x=normalized_pip[0], y=normalized_pip[1], z=normalized_pip[2])
                ring_mcp = points[RING_FINGER_MCP]
                normalized_mcp = (ring_mcp[0] / image.shape[1], ring_mcp[1] / image.shape[0], ring_mcp[2] / image.shape[2])
                unity_comms.SetPosition_ring_mcp(x=normalized_mcp[0], y=normalized_mcp[1], z=normalized_mcp[2])
                pinky_tip = points[PINKY_TIP]
                normalized_tip = (pinky_tip[0] / image.shape[1], pinky_tip[1] / image.shape[0], pinky_tip[2] / image.shape[2])
                unity_comms.SetPosition_pinky_tip(x=normalized_tip[0], y=normalized_tip[1], z=normalized_tip[2])
                pinky_dip = points[PINKY_DIP]
                normalized_dip = (pinky_dip[0] / image.shape[1], pinky_dip[1] / image.shape[0], pinky_dip[2] / image.shape[2])
                unity_comms.SetPosition_pinky_dip(x=normalized_dip[0], y=normalized_dip[1], z=normalized_dip[2])
                pinky_pip = points[PINKY_PIP]
                normalized_pip = (pinky_pip[0] / image.shape[1], pinky_pip[1] / image.shape[0], pinky_pip[2] / image.shape[2])
                unity_comms.SetPosition_pinky_pip(x=normalized_pip[0], y=normalized_pip[1], z=normalized_pip[2])
                pinky_mcp = points[PINKY_MCP]
                normalized_mcp = (pinky_mcp[0] / image.shape[1], pinky_mcp[1] / image.shape[0], pinky_mcp[2] / image.shape[2])
                unity_comms.SetPosition_pinky_mcp(x=normalized_mcp[0], y=normalized_mcp[1], z=normalized_mcp[2])
                wrist = points[WRIST]
                normalized_wrist = (wrist[0] / image.shape[1], wrist[1] / image.shape[0], wrist[2] / image.shape[2])
                unity_comms.SetPosition_palm(x=normalized_wrist[0], y=normalized_wrist[1], z=normalized_wrist[2])
                thumb_tip = points[THUMB_TIP]
                normalized_tip = (thumb_tip[0] / image.shape[1], thumb_tip[1] / image.shape[0], thumb_tip[2] / image.shape[2])
                unity_comms.SetPosition_thumb_tip(x=normalized_tip[0], y=normalized_tip[1], z=normalized_tip[2])
                thumb_dip = points[THUMB_IP]
                normalized_dip = (thumb_dip[0] / image.shape[1], thumb_dip[1] / image.shape[0], thumb_dip[2] / image.shape[2])
                unity_comms.SetPosition_thumb_dip(x=normalized_dip[0], y=normalized_dip[1], z=normalized_dip[2])
                thumb_mcp = points[THUMB_MCP]
                normalized_mcp = (thumb_mcp[0] / image.shape[1], thumb_mcp[1] / image.shape[0], thumb_mcp[2] / image.shape[2])
                unity_comms.SetPosition_thumb_mcp(x=normalized_mcp[0], y=normalized_mcp[1], z=normalized_mcp[2])

    # # Flip the image horizontally again for normal display.
    # image = cv2.flip(image, 1)

    display_image = cv2.cvtColor(display_image, cv2.COLOR_RGB2BGR)

    # Display the image with hand landmarks.
    cv2.imshow('MediaPipe Hands', display_image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break



In [ ]:
# Release the video capture and close all windows.
cap.release()
cv2.destroyAllWindows()


In [ ]:
# stop
def is_open_palm(points):
    if points[THUMB_TIP][1] < points[THUMB_IP][1] and \
        points[INDEX_FINGER_TIP][1] < points[INDEX_FINGER_PIP][1] and \
        points[MIDDLE_FINGER_TIP][1] < points[MIDDLE_FINGER_PIP][1] and \
        points[RING_FINGER_TIP][1] < points[RING_FINGER_PIP][1] and \
        points[PINKY_TIP][1] < points[PINKY_PIP][1]:
        return True
    else:
        return False
# to go forward
def is_index_finger_extended(points):
    if points[INDEX_FINGER_TIP][1] < points[INDEX_FINGER_DIP][1] and \
        points[INDEX_FINGER_DIP][1] < points[INDEX_FINGER_PIP][1] and \
        points[INDEX_FINGER_PIP][1] < points[INDEX_FINGER_MCP][1] and \
        points[INDEX_FINGER_MCP][1] < points[WRIST][1]:
        if is_open_palm(points) == True:
            return False
        elif is_index_and_middle_finger_extended(points) == True:
            return False
        else:
            return True
    else:
        return False
    


# to go forward and left
def is_index_and_middle_finger_extended(points):
    if points[INDEX_FINGER_TIP][1] < points[INDEX_FINGER_DIP][1] and \
        points[INDEX_FINGER_DIP][1] < points[INDEX_FINGER_PIP][1] and \
        points[INDEX_FINGER_PIP][1] < points[INDEX_FINGER_MCP][1] and \
        points[INDEX_FINGER_MCP][1] < points[WRIST][1] and \
        points[MIDDLE_FINGER_TIP][1] < points[MIDDLE_FINGER_DIP][1] and \
        points[MIDDLE_FINGER_DIP][1] < points[MIDDLE_FINGER_PIP][1] and \
        points[MIDDLE_FINGER_PIP][1] < points[MIDDLE_FINGER_MCP][1] and \
        points[MIDDLE_FINGER_MCP][1] < points[WRIST][1]:
        if is_open_palm(points):
            return False
        elif is_index_middle_ring_finger_extended(points):
            return False
        else:
            return True
    return False

#go forward and right
def is_index_middle_ring_finger_extended(points):
    if points[INDEX_FINGER_TIP][1] < points[INDEX_FINGER_DIP][1] and \
        points[INDEX_FINGER_DIP][1] < points[INDEX_FINGER_PIP][1] and \
        points[INDEX_FINGER_PIP][1] < points[INDEX_FINGER_MCP][1] and \
        points[INDEX_FINGER_MCP][1] < points[WRIST][1] and \
        points[MIDDLE_FINGER_TIP][1] < points[MIDDLE_FINGER_DIP][1] and \
        points[MIDDLE_FINGER_DIP][1] < points[MIDDLE_FINGER_PIP][1] and \
        points[MIDDLE_FINGER_PIP][1] < points[MIDDLE_FINGER_MCP][1] and \
        points[MIDDLE_FINGER_MCP][1] < points[WRIST][1] and \
        points[RING_FINGER_TIP][1] < points[RING_FINGER_DIP][1] and \
        points[RING_FINGER_DIP][1] < points[RING_FINGER_PIP][1] and \
        points[RING_FINGER_PIP][1] < points[RING_FINGER_MCP][1] and \
        points[RING_FINGER_MCP][1] < points[WRIST][1]:
        if is_open_palm(points):
            return False
        else:
            return True
    return False


# to go backward 
def is_palm_half_closed(points):
    if points[THUMB_TIP][1] < points[THUMB_IP][1] and \
        points[INDEX_FINGER_TIP][1] > points[INDEX_FINGER_PIP][1] and \
        points[MIDDLE_FINGER_TIP][1] > points[MIDDLE_FINGER_PIP][1] and \
        points[RING_FINGER_TIP][1] > points[RING_FINGER_PIP][1] and \
        points[PINKY_TIP][1] > points[PINKY_PIP][1]:
        return True
    else:
        return False

#reset
def is_index_palm_tip_touch(points):# there checking if index  finger tip is lower than thumb tip
    if is_palm_half_closed(points) == False and \
    points[INDEX_FINGER_TIP][1] > points[THUMB_TIP][1]:
        return True
    else:
        return False
